In [19]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd

from bson.json_util import dumps, loads
from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "changeme"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame.from_records(shelter.readAll({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')



image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # customer image location, can you put it in different location and resize?
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    # Button location at top
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                          html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
                  ]),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),

])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-interactivity',"data"), 
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
              ])
def on_click(bt1,bt2):
    # start case
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Cat"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog"})))
    
        
    return df.to_dict('records')




app
